In [1]:
import cv2
import numpy as np
import socket
import struct
import mediapipe as mp
import threading
import time
from tensorflow.keras.models import load_model
import socket
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy import stats
from tensorflow.keras.models import load_model
import threading
import time
import queue
from pydub import AudioSegment
from playsound import playsound
import os

# Hàm để vẽ và lấy keypoints
def draw_keypoints(image, keypoints, keypoint_ids):
    tmp_lst = []
    for idx, keypoint in enumerate(keypoints):
        if idx in keypoint_ids:
            x = int(keypoint.x * image.shape[1])
            y = int(keypoint.y * image.shape[0])
            tmp_lst.append(x)
            tmp_lst.append(y)
    return tmp_lst

def visualize(res):
    global frame1
    # Convert the data to percentages
    percentages = np.array(res).flatten() * 100

    # Colors for the bars: blue if >90% else purple
    colors = [(0, 255, 0) if value > 90 else (128, 0, 128) for value in percentages]

    # Create a white frame
    frame1 = np.ones((480, 720, 3), dtype=np.uint8) * 255

    # Dimensions for the bar chart
    bar_height = 30
    spacing = 15
    max_width = 650  # max width for the bars
    left_margin = 50  # left margin position
    num_bars = len(res[0])  # number of bars

    # Generate random names for demonstration
    names = ['Goi dien', 'Sua', 'Dat nuoc', 'Doi bung', 'Ten',  'Gao', 'Chap nhan', 'Cam on', 'Giup do', 'Tim kiem']

    # Draw each horizontal bar, name, and the segmented 100% mark
    for i, (percentage, name) in enumerate(zip(percentages, names)):
        # Calculate the width and position of each bar
        bar_width = int((percentage / 100) * max_width)
        y_position = i * (bar_height + spacing) + 50

        # Draw the bar
        cv2.rectangle(frame1, 
                      (left_margin, y_position), 
                      (left_margin + bar_width, y_position + bar_height), 
                      colors[i], 
                      -1)

        # Add horizontal name on the bar
        text_position = (left_margin + 5, y_position + bar_height - 10)
        cv2.putText(frame1, name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 
                    0.4, (0, 0, 0), 1, cv2.LINE_AA)

    # Draw the segmented 100% mark
    segment_height = int(420 / num_bars)  # height of each segment
    for i in range(num_bars):
        segment_y_position = 50 + i * (segment_height + spacing)
        cv2.line(frame1, (left_margin + max_width, segment_y_position), 
                 (left_margin + max_width, segment_y_position + segment_height), (0, 0, 0), 1)

    cv2.putText(frame1, '100%', (left_margin + max_width - 30, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, (0, 0, 0), 1, cv2.LINE_AA)
    
    # Add x and y axis labels
    cv2.putText(frame1, 'Percentage (%)', (600, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame1, 'ID', (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, (0, 0, 0), 1, cv2.LINE_AA)

    return frame1

def detect_and_visualize(model, sequence, result_queue):
    global res 
    res = model.predict(sequence, verbose=0)
    result_queue.put(res)

def play_audio(audio_id):
    id_to_filename = {
                        0: '017.mp3',
                        1: '021.mp3',
                        2: '026.mp3',
                        3: '033.mp3',
                        4: '039.mp3',
                        5: '044.mp3',
                        6: '050.mp3',
                        7: '051.mp3',
                        8: '056.mp3',
                        9: '064.mp3'
                    }
    
    folder_path = 'Voices'
    
    if audio_id in id_to_filename:
        audio_file_name = id_to_filename[audio_id]
        audio_file_path = os.path.join(folder_path, audio_file_name)
        
        if os.path.isfile(audio_file_path):
            playsound(audio_file_path)
        else:
            print(f"The file {audio_file_name} does not exist.")
    else:
        print("Invalid ID. Please enter an ID between 0 and 9.")

# Set mediapipe model 
def process_webcam_stream(model):
    #initial value
    max_length = 120
    keypoint_ids = [15, 17, 19, 21, 16, 18, 20, 22, 0, 2, 5]
    # khởi tạo socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('0.0.0.0', 123))
    server_socket.listen(10)
    conn, addr = server_socket.accept()
    print(f"Đã kết nối với client tại địa chỉ: {addr}")
    #khởi tạo nhận data
    data = b""
    payload_size = struct.calcsize("L")

    # Khởi tạo MediaPipe Pose
    mp_pose = mp.solutions.pose
    mpDraw = mp.solutions.drawing_utils
    pose = mp_pose.Pose()
    #khoi tao queue
    result_queue = queue.Queue()
    # 1. New detection variables
    predictions = {key: 0 for key in range(10)}
    sequence = []
    threshold = 0.985
    step_count = 0
    max_step_count = 10      #khoảng cách giữa 2 lần detect
    fps_count = 0
    time_count = 0
    frame1 = np.ones((480, 720, 3), dtype=np.uint8) * 255
    send_prediction = 404    #biến tin nhắn mặc định được gửi đi
    send_count = 0   #biến đềm thời gian giữa 2 lần gửi tin nhắn
    delay_detect_frame = 0

    while True:
        start = time.time()
        while len(data) < payload_size:
            data += conn.recv(4096)

        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("L", packed_msg_size)[0]

        while len(data) < msg_size:
            data += conn.recv(4096)

        frame_data = data[:msg_size]
        data = data[msg_size:]
        frame = np.frombuffer(frame_data, dtype=np.uint8)
        frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
        # Xoay frame 90 độ về bên trái
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 0)
        height, width = frame.shape[:2]
        frame = cv2.resize(frame, (960, 1080))

        # Xử lý frame để trích xuất keypoints
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            mpDraw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            keypoints = draw_keypoints(frame, results.pose_landmarks.landmark, keypoint_ids)
            sequence.append(keypoints)
            step_count += 1
            if len(sequence) >= max_length and step_count > max_step_count and delay_detect_frame == 0:   
                sequence = [sequence[-max_length:]]   
                # predict
                t1 = threading.Thread(target = detect_and_visualize, args = (model, [sequence], result_queue))
                t1.start()  

                if not result_queue.empty():
                    res = result_queue.get()
                    # print(res)
                    # t2 = threading.Thread(target = visualize, args = (res))
                    # t2.start()
                    if res[0][np.argmax(res)] > threshold:
                        predictions[np.argmax(res)] += 1
                        if predictions[np.argmax(res)] == 3:
                            print(np.argmax(res))
                            send_prediction = np.argmax(res)
                            delay_detect_frame = 60
                            predictions = {key: 0 for key in predictions}
                            t2 = threading.Thread(target = play_audio, args = (int(np.argmax(res)),))
                            t2.start()
                    frame1 = visualize(res)

                step_count = 0

        if delay_detect_frame > 0:
            delay_detect_frame -= 1
        else:
            send_prediction = 404

        if send_count == 10:
            result = str(send_prediction).zfill(3).encode()
            conn.sendall(result)
            send_count = 0
        else:
            send_count += 1
        
        frame = cv2.resize(frame, (width, height))
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)
        cv2.imshow("Visualization", frame1)

        #calculate FPS
        time_count += time.time() - start
        if fps_count == 30:
            print(f"FPS: {1/(time_count/ 30):.2f}")
            time_count = 0
            fps_count = 0
        fps_count += 1

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [2]:
model = load_model(r"model\model_checkpoint_4.h5")

# Gọi hàm xử lý video từ webcam
process_webcam_stream(model)

Đã kết nối với client tại địa chỉ: ('192.168.1.4', 54460)
FPS: 21.13
FPS: 28.76
FPS: 28.63
FPS: 28.36
FPS: 30.21
FPS: 29.36
FPS: 29.47
FPS: 29.64
FPS: 29.13
FPS: 30.33
FPS: 29.38
FPS: 28.74
FPS: 29.24
FPS: 28.79
FPS: 29.94
FPS: 28.65
FPS: 30.00
FPS: 29.53
FPS: 30.00
FPS: 28.93
FPS: 30.18


NameError: name 'cap' is not defined

: 